In [ ]:
import sys
import os

PACKAGE_PARENT = '../../../'
sys.path.append(PACKAGE_PARENT)

from alphaslime.agents.RL.policygrad.torch.ppo import PPOAgent

from alphaslime.trainer.trainerSA import TrainerSA
from alphaslime.store.constantConfig import Constants
from alphaslime.store.config import Config



from pandas import read_csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import slimevolleygym
import gym
import csv

import time
import torch

import random

In [ ]:
# create directory if not present
if not os.path.exists('tmp/ppo/'):
    os.makedirs('tmp/ppo/')

if not os.path.exists('plots/'):
    os.makedirs('plots/')

In [ ]:
# constant config
env_id = "SlimeVolley-v0"
env = gym.make(env_id)

# actions for slimeball
action_table = [[0, 0, 0], # NOOP
                [1, 0, 0], # LEFT (forward)
                [1, 0, 1], # UPLEFT (forward jump)
                [0, 0, 1], # UP (jump)
                [0, 1, 1], # UPRIGHT (backward jump)
                [0, 1, 0]] # RIGHT (backward)

# env.seed(256)
# random.seed(256)


# agent config
STEP_UPDATE = 4096
input_dims = env.observation_space.shape
gamma = 0.99
alpha = 0.0003
gae_lambda = 0.95
policy_clip = 0.2
batch_size = 64
n_epochs = 10


# training config
threshold = 195
is_threshold_stop = False
running_avg_len = 100
is_progress = True
EPISODES = 300
EPISODES = 1000

In [ ]:
const = {
    'env': env,
    'action_table': action_table
}



agent_config = {
    'input_dims': input_dims,
    'gamma': gamma,
    'alpha':alpha,
    'gae_lambda': gae_lambda,
    'policy_clip': policy_clip,
    'batch_size': batch_size,
    'n_epochs': n_epochs,
    'STEP_UPDATE': STEP_UPDATE,
    'verbose': False
}


training_configs = {
    'agent_type': PPOAgent,
    'EPISODES': EPISODES,
    'is_progress': is_progress,
    'threshold': threshold, 
    'is_threshold_stop': is_threshold_stop,
    'running_avg_len': running_avg_len
}


CONST = Constants(const)
agent_hyper = Config(agent_config)
agent_training_configs = Config(training_configs)

figure_file = 'plots/cartpole.png'

In [ ]:
def plot_learning_curve_plus_score(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.plot(scores)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)


In [ ]:
agent = PPOAgent(CONSTANTS=CONST, config=agent_config)
# train agent
agent.train(agent_training_configs)

training_data = agent.get_training_data()

score_history = training_data['rewards']

In [ ]:
x = [i+1 for i in range(len(score_history))]
# plot_learning_curve(x, score_history, figure_file)
plot_learning_curve_plus_score(x, score_history, figure_file)